In [1]:
import sys
    # caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'D:/books/python/0.   Fluent Python, 2nd Edition/example-code-2e/16-op-overloading/')
sys.path

['D:\\books\\python\\0.   Fluent Python, 2nd Edition',
 'D:/books/python/0.   Fluent Python, 2nd Edition/example-code-2e/16-op-overloading/',
 'C:\\Users\\lidan\\miniconda3\\python38.zip',
 'C:\\Users\\lidan\\miniconda3\\DLLs',
 'C:\\Users\\lidan\\miniconda3\\lib',
 'C:\\Users\\lidan\\miniconda3',
 '',
 'C:\\Users\\lidan\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\magic_impute-2.0.4-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\seqc-0.2.0-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\weasyprint-56.1-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\cairocffi-1.3.0-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\win32',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\Pythonwin']

In [2]:
"""
A two-dimensional vector class

    >>> v1 = Vector2d(3, 4)
    >>> print(v1.x, v1.y)
    3.0 4.0
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector2d(3.0, 4.0)
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector2d(0, 0))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector2d.frombytes(bytes(v1))
    >>> v1_clone
    Vector2d(3.0, 4.0)
    >>> v1 == v1_clone
    True


Tests of ``format()`` with Cartesian coordinates:

    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of the ``angle`` method::

    >>> Vector2d(0, 0).angle()
    0.0
    >>> Vector2d(1, 0).angle()
    0.0
    >>> epsilon = 10**-8
    >>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon
    True
    >>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon
    True


Tests of ``format()`` with polar coordinates:

    >>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector2d(1, 1), '.3ep')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector2d(1, 1), '0.5fp')
    '<1.41421, 0.78540>'


Tests of `x` and `y` read-only properties:

    >>> v1.x, v1.y
    (3.0, 4.0)
    >>> v1.x = 123
    Traceback (most recent call last):
      ...
    AttributeError: can't set attribute 'x'


Tests of hashing:

    >>> v1 = Vector2d(3, 4)
    >>> v2 = Vector2d(3.1, 4.2)
    >>> hash(v1), hash(v2)
    (7, 384307168202284039)
    >>> len({v1, v2})
    2

"""

from array import array
import math

class Vector2d:
    __match_args__ = ('x', 'y')

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)


In [3]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [4]:
x, y = v1
x, y

(3.0, 4.0)

In [5]:
v1

Vector2d(3.0, 4.0)

In [8]:
repr(v1)

'Vector2d(3.0, 4.0)'

In [9]:
help(eval)

Help on built-in function eval in module builtins:

eval(source, globals=None, locals=None, /)
    Evaluate the given source in the context of globals and locals.
    
    The source may be a string representing a Python expression
    or a code object as returned by compile().
    The globals must be a dictionary and locals can be any mapping,
    defaulting to the current globals and locals.
    If only globals is given, locals defaults to it.



In [12]:
help(repr)

Help on built-in function repr in module builtins:

repr(obj, /)
    Return the canonical string representation of the object.
    
    For many object types, including most builtins, eval(repr(obj)) == obj.



In [11]:
eval('v1')

Vector2d(3.0, 4.0)

In [7]:
eval(repr(v1))

Vector2d(3.0, 4.0)

In [6]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [13]:
print(v1)

(3.0, 4.0)


In [14]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [15]:
abs(v1)

5.0

In [16]:
Vector2d.frombytes(bytes(v1))

Vector2d(3.0, 4.0)

In [17]:
format(v1)

'(3.0, 4.0)'

In [18]:
format(v1, '.2f')

'(3.00, 4.00)'

In [19]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

In [20]:
Vector2d(0, 0).angle()

0.0

In [21]:
format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS

'<1.4142135623730951, 0.7853981633974483>'

In [22]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [26]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2) 
hash(v1)

7

In [27]:
hash(v2)

384307168202284039

In [28]:
len({v1, v2})

2

In [29]:
print(v1)

(3.0, 4.0)


In [30]:
str(tuple(v1))

'(3.0, 4.0)'

In [32]:
v1.typecode

'd'

In [33]:
help(ord)

Help on built-in function ord in module builtins:

ord(c, /)
    Return the Unicode code point for a one-character string.



In [31]:
ord(v1.typecode)

100

In [36]:
bytes(v1)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [37]:
len(bytes(v1))

17

In [43]:
[byte for byte in bytes(v1)]

[100, 0, 0, 0, 0, 0, 0, 8, 64, 0, 0, 0, 0, 0, 0, 16, 64]

In [42]:
print(bytes(v1)[0])

100


In [34]:
bytes(v1)[0]

100

In [35]:
bytes(v1)[1]

0

In [44]:
v1[0]

TypeError: 'Vector2d' object is not subscriptable

In [45]:
"""
A multi-dimensional ``Vector`` class, take 6: operator ``+``

A ``Vector`` is built from an iterable of numbers::

    >>> Vector([3.1, 4.2])
    Vector([3.1, 4.2])
    >>> Vector((3, 4, 5))
    Vector([3.0, 4.0, 5.0])
    >>> Vector(range(10))
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])


Tests with 2-dimensions (same results as ``vector2d_v1.py``)::

    >>> v1 = Vector([3, 4])
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector([3.0, 4.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector([0, 0]))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0])
    >>> v1 == v1_clone
    True


Tests with 3-dimensions::

    >>> v1 = Vector([3, 4, 5])
    >>> x, y, z = v1
    >>> x, y, z
    (3.0, 4.0, 5.0)
    >>> v1
    Vector([3.0, 4.0, 5.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0, 5.0)
    >>> abs(v1)  # doctest:+ELLIPSIS
    7.071067811...
    >>> bool(v1), bool(Vector([0, 0, 0]))
    (True, False)


Tests with many dimensions::

    >>> v7 = Vector(range(7))
    >>> v7
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])
    >>> abs(v7)  # doctest:+ELLIPSIS
    9.53939201...


Test of ``.__bytes__`` and ``.frombytes()`` methods::

    >>> v1 = Vector([3, 4, 5])
    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0, 5.0])
    >>> v1 == v1_clone
    True


Tests of sequence behavior::

    >>> v1 = Vector([3, 4, 5])
    >>> len(v1)
    3
    >>> v1[0], v1[len(v1)-1], v1[-1]
    (3.0, 5.0, 5.0)


Test of slicing::

    >>> v7 = Vector(range(7))
    >>> v7[-1]
    6.0
    >>> v7[1:4]
    Vector([1.0, 2.0, 3.0])
    >>> v7[-1:]
    Vector([6.0])
    >>> v7[1,2]
    Traceback (most recent call last):
      ...
    TypeError: 'tuple' object cannot be interpreted as an integer


Tests of dynamic attribute access::

    >>> v7 = Vector(range(10))
    >>> v7.x
    0.0
    >>> v7.y, v7.z, v7.t
    (1.0, 2.0, 3.0)

Dynamic attribute lookup failures::

    >>> v7.k
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'k'
    >>> v3 = Vector(range(3))
    >>> v3.t
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 't'
    >>> v3.spam
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'spam'


Tests of hashing::

    >>> v1 = Vector([3, 4])
    >>> v2 = Vector([3.1, 4.2])
    >>> v3 = Vector([3, 4, 5])
    >>> v6 = Vector(range(6))
    >>> hash(v1), hash(v3), hash(v6)
    (7, 2, 1)


Most hash codes of non-integers vary from a 32-bit to 64-bit Python build::

    >>> import sys
    >>> hash(v2) == (384307168202284039 if sys.maxsize > 2**32 else 357915986)
    True


Tests of ``format()`` with Cartesian coordinates in 2D::

    >>> v1 = Vector([3, 4])
    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of ``format()`` with Cartesian coordinates in 3D and 7D::

    >>> v3 = Vector([3, 4, 5])
    >>> format(v3)
    '(3.0, 4.0, 5.0)'
    >>> format(Vector(range(7)))
    '(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'


Tests of ``format()`` with spherical coordinates in 2D, 3D and 4D::

    >>> format(Vector([1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector([1, 1]), '.3eh')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector([1, 1]), '0.5fh')
    '<1.41421, 0.78540>'
    >>> format(Vector([1, 1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.73205..., 0.95531..., 0.78539...>'
    >>> format(Vector([2, 2, 2]), '.3eh')
    '<3.464e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 0, 0]), '0.5fh')
    '<0.00000, 0.00000, 0.00000>'
    >>> format(Vector([-1, -1, -1, -1]), 'h')  # doctest:+ELLIPSIS
    '<2.0, 2.09439..., 2.18627..., 3.92699...>'
    >>> format(Vector([2, 2, 2, 2]), '.3eh')
    '<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 1, 0, 0]), '0.5fh')
    '<1.00000, 1.57080, 0.00000, 0.00000>'


Unary operator tests::

    >>> v1 = Vector([3, 4])
    >>> abs(v1)
    5.0
    >>> -v1
    Vector([-3.0, -4.0])
    >>> +v1
    Vector([3.0, 4.0])


Basic tests of operator ``+``::

    >>> v1 = Vector([3, 4, 5])
    >>> v2 = Vector([6, 7, 8])
    >>> v1 + v2
    Vector([9.0, 11.0, 13.0])
    >>> v1 + v2 == Vector([3+6, 4+7, 5+8])
    True
    >>> v3 = Vector([1, 2])
    >>> v1 + v3  # short vectors are filled with 0.0 on addition
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with mixed types::

    >>> v1 + (10, 20, 30)
    Vector([13.0, 24.0, 35.0])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> v1 + v2d
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with mixed types, swapped operands::

    >>> (10, 20, 30) + v1
    Vector([13.0, 24.0, 35.0])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> v2d + v1
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with an unsuitable operand:

    >>> v1 + 1
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'Vector' and 'int'
    >>> v1 + 'ABC'
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'Vector' and 'str'
"""

from array import array
import reprlib
import math
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

# tag::VECTOR_V6_UNARY[]
    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)  # <1>

    def __pos__(self):
        return Vector(self)  # <2>
# end::VECTOR_V6_UNARY[]

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# tag::VECTOR_V6_ADD[]
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other
# end::VECTOR_V6_ADD[]


In [46]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [47]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [48]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [49]:
v7 = Vector(range(7))
v7[-1]

6.0

In [50]:
v7[3:6]

Vector([3.0, 4.0, 5.0])

In [51]:
v7[1:2]

Vector([1.0])

In [52]:
v7[1,2]

TypeError: 'tuple' object cannot be interpreted as an integer

In [53]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [56]:
v2 + v1

Vector([9.0, 11.0, 13.0])

In [54]:
v1 + v2 == Vector([3+6, 4+7, 5+8])

True

In [55]:
v3 = Vector([1, 2])
v1 + v3 

Vector([4.0, 6.0, 5.0])

In [57]:
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [58]:
from vector2d_v3 import Vector2d
v2d = Vector2d(1, 2)
v1 + v2d

Vector([4.0, 6.0, 5.0])

In [59]:
v2d + v1

Vector([4.0, 6.0, 5.0])

In [60]:
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [62]:
"""
A multi-dimensional ``Vector`` class, take 7: operator ``*``

A ``Vector`` is built from an iterable of numbers::

    >>> Vector([3.1, 4.2])
    Vector([3.1, 4.2])
    >>> Vector((3, 4, 5))
    Vector([3.0, 4.0, 5.0])
    >>> Vector(range(10))
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])


Tests with 2-dimensions (same results as ``vector2d_v1.py``)::

    >>> v1 = Vector([3, 4])
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector([3.0, 4.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector([0, 0]))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0])
    >>> v1 == v1_clone
    True


Tests with 3-dimensions::

    >>> v1 = Vector([3, 4, 5])
    >>> x, y, z = v1
    >>> x, y, z
    (3.0, 4.0, 5.0)
    >>> v1
    Vector([3.0, 4.0, 5.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0, 5.0)
    >>> abs(v1)  # doctest:+ELLIPSIS
    7.071067811...
    >>> bool(v1), bool(Vector([0, 0, 0]))
    (True, False)


Tests with many dimensions::

    >>> v7 = Vector(range(7))
    >>> v7
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])
    >>> abs(v7)  # doctest:+ELLIPSIS
    9.53939201...


Test of ``.__bytes__`` and ``.frombytes()`` methods::

    >>> v1 = Vector([3, 4, 5])
    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0, 5.0])
    >>> v1 == v1_clone
    True


Tests of sequence behavior::

    >>> v1 = Vector([3, 4, 5])
    >>> len(v1)
    3
    >>> v1[0], v1[len(v1)-1], v1[-1]
    (3.0, 5.0, 5.0)


Test of slicing::

    >>> v7 = Vector(range(7))
    >>> v7[-1]
    6.0
    >>> v7[1:4]
    Vector([1.0, 2.0, 3.0])
    >>> v7[-1:]
    Vector([6.0])
    >>> v7[1,2]
    Traceback (most recent call last):
      ...
    TypeError: 'tuple' object cannot be interpreted as an integer


Tests of dynamic attribute access::

    >>> v7 = Vector(range(10))
    >>> v7.x
    0.0
    >>> v7.y, v7.z, v7.t
    (1.0, 2.0, 3.0)

Dynamic attribute lookup failures::

    >>> v7.k
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'k'
    >>> v3 = Vector(range(3))
    >>> v3.t
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 't'
    >>> v3.spam
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'spam'


Tests of hashing::

    >>> v1 = Vector([3, 4])
    >>> v2 = Vector([3.1, 4.2])
    >>> v3 = Vector([3, 4, 5])
    >>> v6 = Vector(range(6))
    >>> hash(v1), hash(v3), hash(v6)
    (7, 2, 1)


Most hash codes of non-integers vary from a 32-bit to 64-bit Python build::

    >>> import sys
    >>> hash(v2) == (384307168202284039 if sys.maxsize > 2**32 else 357915986)
    True


Tests of ``format()`` with Cartesian coordinates in 2D::

    >>> v1 = Vector([3, 4])
    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of ``format()`` with Cartesian coordinates in 3D and 7D::

    >>> v3 = Vector([3, 4, 5])
    >>> format(v3)
    '(3.0, 4.0, 5.0)'
    >>> format(Vector(range(7)))
    '(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'


Tests of ``format()`` with spherical coordinates in 2D, 3D and 4D::

    >>> format(Vector([1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector([1, 1]), '.3eh')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector([1, 1]), '0.5fh')
    '<1.41421, 0.78540>'
    >>> format(Vector([1, 1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.73205..., 0.95531..., 0.78539...>'
    >>> format(Vector([2, 2, 2]), '.3eh')
    '<3.464e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 0, 0]), '0.5fh')
    '<0.00000, 0.00000, 0.00000>'
    >>> format(Vector([-1, -1, -1, -1]), 'h')  # doctest:+ELLIPSIS
    '<2.0, 2.09439..., 2.18627..., 3.92699...>'
    >>> format(Vector([2, 2, 2, 2]), '.3eh')
    '<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 1, 0, 0]), '0.5fh')
    '<1.00000, 1.57080, 0.00000, 0.00000>'


Unary operator tests::

    >>> v1 = Vector([3, 4])
    >>> abs(v1)
    5.0
    >>> -v1
    Vector([-3.0, -4.0])
    >>> +v1
    Vector([3.0, 4.0])


Basic tests of operator ``+``::

    >>> v1 = Vector([3, 4, 5])
    >>> v2 = Vector([6, 7, 8])
    >>> v1 + v2
    Vector([9.0, 11.0, 13.0])
    >>> v1 + v2 == Vector([3+6, 4+7, 5+8])
    True
    >>> v3 = Vector([1, 2])
    >>> v1 + v3  # short vectors are filled with 0.0 on addition
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with mixed types::

    >>> v1 + (10, 20, 30)
    Vector([13.0, 24.0, 35.0])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> v1 + v2d
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with mixed types, swapped operands::

    >>> (10, 20, 30) + v1
    Vector([13.0, 24.0, 35.0])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> v2d + v1
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with an unsuitable operand:

    >>> v1 + 1
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'Vector' and 'int'
    >>> v1 + 'ABC'
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'Vector' and 'str'


Basic tests of operator ``*``::

    >>> v1 = Vector([1, 2, 3])
    >>> v1 * 10
    Vector([10.0, 20.0, 30.0])
    >>> 10 * v1
    Vector([10.0, 20.0, 30.0])


Tests of ``*`` with unusual but valid operands::

    >>> v1 * True
    Vector([1.0, 2.0, 3.0])
    >>> from fractions import Fraction
    >>> v1 * Fraction(1, 3)  # doctest:+ELLIPSIS
    Vector([0.3333..., 0.6666..., 1.0])


Tests of ``*`` with unsuitable operands::

    >>> v1 * (1, 2)
    Traceback (most recent call last):
    ...
    TypeError: can't multiply sequence by non-int of type 'Vector'


Tests of ``@``::

    >>> va = Vector([1, 2, 3])
    >>> vz = Vector([5, 6, 7])
    >>> va @ vz == 38.0  # 1*5 + 2*6 + 3*7
    True
    >>> [10, 20, 30] @ vz
    380.0
    >>> va @ 3
    Traceback (most recent call last):
    ...
    TypeError: unsupported operand type(s) for @: 'Vector' and 'int'


For ``@`` to work, both operands need to have the same length::

    >>> va = Vector([1, 2, 3])
    >>> vb = Vector([1, 2])
    >>> va @ vb
    Traceback (most recent call last):
    ...
    ValueError: @ requires vectors of equal length.

"""

from array import array
import reprlib
import math
import functools
import operator
import itertools
from collections import abc


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar):
        try:
            factor = float(scalar)
        except TypeError:
            return NotImplemented
        return Vector(n * factor for n in self)

    def __rmul__(self, scalar):
        return self * scalar

    def __matmul__(self, other):
        if (isinstance(other, abc.Sized) and
            isinstance(other, abc.Iterable)):
            if len(self) == len(other):
                return sum(a * b for a, b in zip(self, other))
            else:
                raise ValueError('@ requires vectors of equal length.')
        else:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other


In [63]:
v1 = Vector([3, 4])
abs(v1)

5.0

In [64]:
-v1

Vector([-3.0, -4.0])

In [65]:
+v1

Vector([3.0, 4.0])

In [66]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [67]:
v3 = Vector([1, 2])
v1 + v3  # short vectors are filled with 0.0 on addition

Vector([4.0, 6.0, 5.0])

In [68]:
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [69]:
(10, 20, 30) + v1

Vector([13.0, 24.0, 35.0])

In [70]:
from vector2d_v3 import Vector2d
v2d = Vector2d(1, 2)
v1 + v2d

Vector([4.0, 6.0, 5.0])

In [71]:
v2d+v1

Vector([4.0, 6.0, 5.0])

In [72]:
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [73]:
v1 = Vector([1, 2, 3])
v1 * 10

Vector([10.0, 20.0, 30.0])

In [74]:
10*v1

Vector([10.0, 20.0, 30.0])

In [75]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [77]:
from fractions import Fraction
v1 * Fraction(1, 3)  # doctest:+ELLIPSIS

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

In [78]:
v1 * (1, 2)

TypeError: can't multiply sequence by non-int of type 'Vector'

In [79]:
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz

38.0

In [80]:
1*5 + 2*6 + 3*7

38

In [81]:
[10, 20, 30] @ vz

380.0

In [82]:
va @ 3

TypeError: unsupported operand type(s) for @: 'Vector' and 'int'

In [84]:
va = Vector([1, 2, 3])
vb = Vector([1, 2])
va @ vb

ValueError: @ requires vectors of equal length.

In [85]:
"""
A multi-dimensional ``Vector`` class, take 8: operator ``==``

A ``Vector`` is built from an iterable of numbers::

    >>> Vector([3.1, 4.2])
    Vector([3.1, 4.2])
    >>> Vector((3, 4, 5))
    Vector([3.0, 4.0, 5.0])
    >>> Vector(range(10))
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])


Tests with 2-dimensions (same results as ``vector2d_v1.py``)::

    >>> v1 = Vector([3, 4])
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector([3.0, 4.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector([0, 0]))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0])
    >>> v1 == v1_clone
    True


Tests with 3-dimensions::

    >>> v1 = Vector([3, 4, 5])
    >>> x, y, z = v1
    >>> x, y, z
    (3.0, 4.0, 5.0)
    >>> v1
    Vector([3.0, 4.0, 5.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0, 5.0)
    >>> abs(v1)  # doctest:+ELLIPSIS
    7.071067811...
    >>> bool(v1), bool(Vector([0, 0, 0]))
    (True, False)


Tests with many dimensions::

    >>> v7 = Vector(range(7))
    >>> v7
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])
    >>> abs(v7)  # doctest:+ELLIPSIS
    9.53939201...


Test of ``.__bytes__`` and ``.frombytes()`` methods::

    >>> v1 = Vector([3, 4, 5])
    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0, 5.0])
    >>> v1 == v1_clone
    True


Tests of sequence behavior::

    >>> v1 = Vector([3, 4, 5])
    >>> len(v1)
    3
    >>> v1[0], v1[len(v1)-1], v1[-1]
    (3.0, 5.0, 5.0)


Test of slicing::

    >>> v7 = Vector(range(7))
    >>> v7[-1]
    6.0
    >>> v7[1:4]
    Vector([1.0, 2.0, 3.0])
    >>> v7[-1:]
    Vector([6.0])
    >>> v7[1,2]
    Traceback (most recent call last):
      ...
    TypeError: 'tuple' object cannot be interpreted as an integer


Tests of dynamic attribute access::

    >>> v7 = Vector(range(10))
    >>> v7.x
    0.0
    >>> v7.y, v7.z, v7.t
    (1.0, 2.0, 3.0)

Dynamic attribute lookup failures::

    >>> v7.k
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'k'
    >>> v3 = Vector(range(3))
    >>> v3.t
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 't'
    >>> v3.spam
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'spam'


Tests of hashing::

    >>> v1 = Vector([3, 4])
    >>> v2 = Vector([3.1, 4.2])
    >>> v3 = Vector([3, 4, 5])
    >>> v6 = Vector(range(6))
    >>> hash(v1), hash(v3), hash(v6)
    (7, 2, 1)


Most hash codes of non-integers vary from a 32-bit to 64-bit Python build::

    >>> import sys
    >>> hash(v2) == (384307168202284039 if sys.maxsize > 2**32 else 357915986)
    True


Tests of ``format()`` with Cartesian coordinates in 2D::

    >>> v1 = Vector([3, 4])
    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of ``format()`` with Cartesian coordinates in 3D and 7D::

    >>> v3 = Vector([3, 4, 5])
    >>> format(v3)
    '(3.0, 4.0, 5.0)'
    >>> format(Vector(range(7)))
    '(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'


Tests of ``format()`` with spherical coordinates in 2D, 3D and 4D::

    >>> format(Vector([1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector([1, 1]), '.3eh')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector([1, 1]), '0.5fh')
    '<1.41421, 0.78540>'
    >>> format(Vector([1, 1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.73205..., 0.95531..., 0.78539...>'
    >>> format(Vector([2, 2, 2]), '.3eh')
    '<3.464e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 0, 0]), '0.5fh')
    '<0.00000, 0.00000, 0.00000>'
    >>> format(Vector([-1, -1, -1, -1]), 'h')  # doctest:+ELLIPSIS
    '<2.0, 2.09439..., 2.18627..., 3.92699...>'
    >>> format(Vector([2, 2, 2, 2]), '.3eh')
    '<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 1, 0, 0]), '0.5fh')
    '<1.00000, 1.57080, 0.00000, 0.00000>'


Unary operator tests::

    >>> v1 = Vector([3, 4])
    >>> abs(v1)
    5.0
    >>> -v1
    Vector([-3.0, -4.0])
    >>> +v1
    Vector([3.0, 4.0])


Basic tests of operator ``+``::

    >>> v1 = Vector([3, 4, 5])
    >>> v2 = Vector([6, 7, 8])
    >>> v1 + v2
    Vector([9.0, 11.0, 13.0])
    >>> v1 + v2 == Vector([3+6, 4+7, 5+8])
    True
    >>> v3 = Vector([1, 2])
    >>> v1 + v3  # short vectors are filled with 0.0 on addition
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with mixed types::

    >>> v1 + (10, 20, 30)
    Vector([13.0, 24.0, 35.0])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> v1 + v2d
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with mixed types, swapped operands::

    >>> (10, 20, 30) + v1
    Vector([13.0, 24.0, 35.0])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> v2d + v1
    Vector([4.0, 6.0, 5.0])


Tests of ``+`` with an unsuitable operand:

    >>> v1 + 1
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'Vector' and 'int'
    >>> v1 + 'ABC'
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'Vector' and 'str'


Basic tests of operator ``*``::

    >>> v1 = Vector([1, 2, 3])
    >>> v1 * 10
    Vector([10.0, 20.0, 30.0])
    >>> 10 * v1
    Vector([10.0, 20.0, 30.0])


Tests of ``*`` with unusual but valid operands::

    >>> v1 * True
    Vector([1.0, 2.0, 3.0])
    >>> from fractions import Fraction
    >>> v1 * Fraction(1, 3)  # doctest:+ELLIPSIS
    Vector([0.3333..., 0.6666..., 1.0])


Tests of ``*`` with unsuitable operands::

    >>> v1 * (1, 2)
    Traceback (most recent call last):
      ...
    TypeError: can't multiply sequence by non-int of type 'Vector'


Tests of operator `==`::

    >>> va = Vector(range(1, 4))
    >>> vb = Vector([1.0, 2.0, 3.0])
    >>> va == vb
    True
    >>> vc = Vector([1, 2])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> vc == v2d
    True
    >>> va == (1, 2, 3)
    False


Tests of operator `!=`::

    >>> va != vb
    False
    >>> vc != v2d
    False
    >>> va != (1, 2, 3)
    True

"""

from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

# tag::VECTOR_V8_EQ[]
    def __eq__(self, other):
        if isinstance(other, Vector):  # <1>
            return (len(self) == len(other) and
                    all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented  # <2>
# end::VECTOR_V8_EQ[]

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented

    def __rmul__(self, scalar):
        return self * scalar


In [86]:
operator.index(2)

2

In [87]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [88]:
v3 = Vector([1, 2])
v1 + v3  # short vectors are filled with 0.0 on addition

Vector([4.0, 6.0, 5.0])

In [89]:
(10, 20, 30) + v1

Vector([13.0, 24.0, 35.0])

In [90]:
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [91]:
v1 = Vector([1, 2, 3])
v1 * 10

Vector([10.0, 20.0, 30.0])

In [92]:
10 * v1

Vector([10.0, 20.0, 30.0])

In [93]:
v1 * (1, 2)

TypeError: can't multiply sequence by non-int of type 'Vector'

In [94]:
va = Vector(range(1, 4))
vb = Vector([1.0, 2.0, 3.0])
va == vb

True

In [95]:
va 

Vector([1.0, 2.0, 3.0])

In [96]:
vb

Vector([1.0, 2.0, 3.0])

In [97]:
vc = Vector([1, 2])
from vector2d_v3 import Vector2d
v2d = Vector2d(1, 2)
vc == v2d

True

In [98]:
vc

Vector([1.0, 2.0])

In [99]:
v2d

Vector2d(1.0, 2.0)

In [100]:
va == (1, 2, 3)

False

In [101]:
isinstance((1, 2, 3), Vector)

False

In [102]:
vc*3

Vector([3.0, 6.0])

In [105]:
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz

TypeError: unsupported operand type(s) for @: 'Vector' and 'Vector'

In [108]:
# BEGIN TOMBOLA_ABC

import abc

class Tombola(abc.ABC):  # <1>

    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(sorted(items))

# END TOMBOLA_ABC



In [110]:
random.SystemRandom()

In [106]:
# BEGIN TOMBOLA_BINGO

import random

from tombola import Tombola


class BingoCage(Tombola):  # <1>

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # <2>
        self._items = []
        self.load(items)  # <3>

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # <4>

    def pick(self):  # <5>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # <7>
        self.pick()

# END TOMBOLA_BINGO


In [111]:
"""
======================
AddableBingoCage tests
======================


Tests for __add__:

# tag::ADDABLE_BINGO_ADD_DEMO[]

    >>> vowels = 'AEIOU'
    >>> globe = AddableBingoCage(vowels)  # <1>
    >>> globe.inspect()
    ('A', 'E', 'I', 'O', 'U')
    >>> globe.pick() in vowels  # <2>
    True
    >>> len(globe.inspect())  # <3>
    4
    >>> globe2 = AddableBingoCage('XYZ')  # <4>
    >>> globe3 = globe + globe2
    >>> len(globe3.inspect())  # <5>
    7
    >>> void = globe + [10, 20]  # <6>
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'


# end::ADDABLE_BINGO_ADD_DEMO[]

Tests for __iadd__:

# tag::ADDABLE_BINGO_IADD_DEMO[]

    >>> globe_orig = globe  # <1>
    >>> len(globe.inspect())  # <2>
    4
    >>> globe += globe2  # <3>
    >>> len(globe.inspect())
    7
    >>> globe += ['M', 'N']  # <4>
    >>> len(globe.inspect())
    9
    >>> globe is globe_orig  # <5>
    True
    >>> globe += 1  # <6>
    Traceback (most recent call last):
      ...
    TypeError: right operand in += must be 'Tombola' or an iterable

# end::ADDABLE_BINGO_IADD_DEMO[]

"""

# tag::ADDABLE_BINGO[]
from tombola import Tombola
from bingo import BingoCage


class AddableBingoCage(BingoCage):  # <1>

    def __add__(self, other):
        if isinstance(other, Tombola):  # <2>
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()  # <3>
        else:
            try:
                other_iterable = iter(other)  # <4>
            except TypeError:  # <5>
                msg = ('right operand in += must be '
                       "'Tombola' or an iterable")
                raise TypeError(msg)
        self.load(other_iterable)  # <6>
        return self  # <7>

# end::ADDABLE_BINGO[]


In [112]:
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)  # <1>
globe.inspect()

('A', 'E', 'I', 'O', 'U')

In [113]:
globe.pick() in vowels  # <2>

True

In [114]:
globe.pick()

'U'

In [115]:
globe.pick()

'O'

In [116]:
len(globe.inspect())  # <3>

2

In [118]:
globe2 = AddableBingoCage('XYZ')  # <4>
globe3 = globe + globe2
len(globe3.inspect())  # <5>

5

In [119]:
void = globe + [10, 20]  # <6>

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [121]:
globe_orig = globe  # <1>
len(globe.inspect())  # <2>

2

In [122]:
globe += globe2  # <3>
len(globe.inspect())

5

In [123]:
globe += ['M', 'N']  # <4>
len(globe.inspect())

7

In [124]:
globe is globe_orig  # <5>

True

In [125]:
globe += 1  # <6>

TypeError: right operand in += must be 'Tombola' or an iterable